In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
model = SLIM_BPR_Cython

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
earlystopping_kwargs =  {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }

EvaluatorHoldout: Ignoring 2593 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
def objective(trial):
    # Optuna suggest params
    topK = trial.suggest_int("topK", 5, 1000)
    symmetric = trial.suggest_categorical("symmetric", [True, False])
    sgd_mode = trial.suggest_categorical("sgd_mode", ["sgd", "adagrad", "adam"])
    epochs = 1500
    lambda_i = trial.suggest_loguniform("lambda_i", 1e-5, 1e-2)
    lambda_j = trial.suggest_loguniform("lambda_j", 1e-5, 1e-2)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
    
    recommender = model(URM_train)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "topK": 7,
    "symmetric": True,
    "sgd_mode": "adam",
    "epochs": 40,
    "lambda_i": 0.00032913664880523404,
    "lambda_j": 0.007236536070943505,
    "learning_rate": 0.00021598859241204376,
}

study_name = "SLIMBPR-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=30)

In [5]:
study.best_params

{'topK': 46,
 'symmetric': True,
 'sgd_mode': 'adagrad',
 'lambda_i': 0.00035732097169170193,
 'lambda_j': 0.004542391418975173,
 'learning_rate': 0.0002291767622519941}

In [7]:
final = model(URM_train_validation)
final.fit(topK=46, symmetric=True, sgd_mode="adagrad", epochs=195, lambda_i=0.00035732097169170193, lambda_j=0.004542391418975173, learning_rate=0.0002291767622519941)
#final.fit(topK=7, symmetric=True, sgd_mode="adam", epochs=40, lambda_i=0.00032913664880523404, lambda_j=0.007236536070943505, learning_rate=0.00021598859241204376)
#final.fit(topK=11, symmetric=True, sgd_mode="sgd", epochs=50, lambda_i=0.0025, lambda_j=0.00025, learning_rate=0.05,)
#final.fit(**study.best_params, epochs=260, random_seed=1234)

SLIM_BPR_Recommender: URM Detected 199 ( 1.6%) users with no interactions.
SLIM_BPR_Recommender: URM Detected 116 ( 0.5%) items with no interactions.
Deallocating Cython objects
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 12778.00 MB (53.46%) of 23904.00 MB, required is 1975.27 MB. Using dense matrix.
Processed 12638 (100.0%) in 0.50 sec. BPR loss is 1.25E-06. Sample per second: 25330
SLIM_BPR_Recommender: Epoch 1 of 195. Elapsed time 0.02 sec
Processed 12638 (100.0%) in 0.52 sec. BPR loss is 4.26E-06. Sample per second: 24138
SLIM_BPR_Recommender: Epoch 2 of 195. Elapsed time 0.05 sec
Processed 12638 (100.0%) in 0.55 sec. BPR loss is 7.89E-06. Sample per second: 22844
SLIM_BPR_Recommender: Epoch 3 of 195. Elapsed time 0.08 sec
Processed 12638 (100.0%) in 0.58 sec. BPR loss is 1.08E-05. Sample per second: 21646
SLIM_BPR_Recommender: Epoch 4 of 195. Elapsed time 0.11 sec
Processed 12638 (100.0%) in 0.61 sec. BPR loss is 1.49E-05. Sample per second: 

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

In [ ]:
## MAP = .0338132